In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import os
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Define path to your dataset
dataset_val_path = "/kaggle/input/bird-25-data/Birds_25/val"
dataset_train_path="/kaggle/input/bird-25-data/Birds_25/train"
dataset_test_path="/kaggle/input/bird-25-data/Birds_25/test"


transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create dataset object
dataset_val = ImageFolder(root=dataset_val_path, transform=transform)
dataset_train = ImageFolder(root=dataset_train_path, transform=transform)
dataset_test = ImageFolder(root=dataset_test_path, transform=transform)

# Create data loader for the entire dataset
train_data_loader = DataLoader(dataset_train, batch_size=32, shuffle=True)
test_data_loader = DataLoader(dataset_test, batch_size=32, shuffle=True)
val_data_loader = DataLoader(dataset_val, batch_size=32, shuffle=True)

# Print the number of classes in the dataset
num_classes = len(dataset_train.classes)
print("Number of classes:", num_classes)


Number of classes: 25


In [7]:
import argparse
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os




import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d  # Default to BatchNorm2d if none specified
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = norm_layer(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = norm_layer(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                norm_layer(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, n, num_classes=25, norm_layer=None):
        super(ResNet, self).__init__()
        self.in_planes = 16

        if norm_layer is None:
            norm_layer = nn.BatchNorm2d  # Default to BatchNorm2d if none specified
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = norm_layer(16)
        
        # Now n determines the number of blocks in each layer
        self.layer1 = self._make_layer(block, 16, n, stride=1, norm_layer=norm_layer)
        self.layer2 = self._make_layer(block, 32, n, stride=2, norm_layer=norm_layer)
        self.layer3 = self._make_layer(block, 64, n, stride=2, norm_layer=norm_layer)
        self.linear = nn.Linear(64*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride, norm_layer):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride, norm_layer))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.adaptive_avg_pool2d(out, (1, 1))
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

def ResNet18(n=2, num_classes=25, norm_layer=None):
    # Adjusting the model creation function to accept n
    return ResNet(BasicBlock, n, num_classes, norm_layer)



class MyNoNorm(nn.Module):
    def __init__(self, num_features=None, *args, **kwargs):
        super(MyNoNorm, self).__init__()
        
        
    def forward(self, x):
        
        return x



class MyLayerNorm(nn.Module):
    def __init__(self, normalized_shape, eps=1e-5):
        super(MyLayerNorm, self).__init__()
        self.eps = eps
        self.gamma = nn.Parameter(torch.ones(normalized_shape))
        self.beta = nn.Parameter(torch.zeros(normalized_shape))

    def forward(self, x):
        mean = x.mean(dim=(1, 2, 3), keepdim=True)
        var = ((x - mean) ** 2).mean(dim=(1, 2, 3), keepdim=True)
        x_normalized = (x - mean) / torch.sqrt(var + self.eps)
        return self.gamma * x_normalized + self.beta

class MyGroupNorm(nn.Module):
    def __init__(self, num_groups, num_channels, eps=1e-5):
        super(MyGroupNorm, self).__init__()
        self.num_groups = num_groups
        self.eps = eps
        self.gamma = nn.Parameter(torch.ones(num_channels))
        self.beta = nn.Parameter(torch.zeros(num_channels))

    def forward(self, x):
        N, C, H, W = x.size()
        G = self.num_groups
        x = x.view(N, G, C // G, H, W)
        mean = x.mean(dim=(2, 3, 4), keepdim=True)
        var = ((x - mean) ** 2).mean(dim=(2, 3, 4), keepdim=True)
        x = (x - mean) / torch.sqrt(var + self.eps)
        x = x.view(N, C, H, W)
        return self.gamma.view(1, C, 1, 1) * x + self.beta.view(1, C, 1, 1)

class MyBatchInstanceNorm2d(nn.Module):
    def __init__(self, num_features, eps=1e-5, momentum=0.1):
        super(MyBatchInstanceNorm2d, self).__init__()
        self.eps = eps
        self.momentum = momentum
        # Batch Norm components
        self.bn_gamma = nn.Parameter(torch.ones(num_features))
        self.bn_beta = nn.Parameter(torch.zeros(num_features))
        # Register running_mean and running_var as buffers
        self.register_buffer('running_mean', torch.zeros(num_features))
        self.register_buffer('running_var', torch.ones(num_features))
        # Instance Norm components
        self.in_gamma = nn.Parameter(torch.ones(num_features))
        self.in_beta = nn.Parameter(torch.zeros(num_features))
        self.num_features = num_features

    def forward(self, x):
        if self.training:
           
            bn_mean = x.mean([0, 2, 3], keepdim=True)
            bn_var = x.var([0, 2, 3], keepdim=True, unbiased=False)
            
            self.running_mean *= (1 - self.momentum)
            self.running_mean += self.momentum * bn_mean.squeeze()
            self.running_var *= (1 - self.momentum)
            self.running_var += self.momentum * bn_var.squeeze()
        else:
            
            bn_mean = self.running_mean.view(1, self.num_features, 1, 1)
            bn_var = self.running_var.view(1, self.num_features, 1, 1)
            
        bn_x = (x - bn_mean) / torch.sqrt(bn_var + self.eps)
        bn_out = self.bn_gamma.view(1, self.num_features, 1, 1) * bn_x + self.bn_beta.view(1, self.num_features, 1, 1)
        
        
        in_mean = x.mean([2, 3], keepdim=True)
        in_var = x.var([2, 3], keepdim=True, unbiased=False)
        in_x = (x - in_mean) / torch.sqrt(in_var + self.eps)
        in_out = self.in_gamma.view(1, self.num_features, 1, 1) * in_x + self.in_beta.view(1, self.num_features, 1, 1)
        
        
        return 0.5 * (bn_out + in_out)


class MyBatchNorm2d(nn.Module):
    def __init__(self, num_features, eps=1e-5, momentum=0.1):
        super(MyBatchNorm2d, self).__init__()
        self.num_features = num_features
        self.eps = eps
        self.momentum = momentum
        self.num_features = num_features
        self.gamma = nn.Parameter(torch.ones(num_features))
        self.beta = nn.Parameter(torch.zeros(num_features))
        self.register_buffer('running_mean', torch.zeros(num_features))
        self.register_buffer('running_var', torch.ones(num_features))

    def forward(self, x):
        if self.training:
            mean = x.mean([0, 2, 3], keepdim=True)
            var = x.var([0, 2, 3], keepdim=True, unbiased=False)
            x_normalized = (x - mean) / torch.sqrt(var + self.eps)
            self.running_mean = (1 - self.momentum) * self.running_mean.detach() + self.momentum * mean.squeeze()
            self.running_var = (1 - self.momentum) * self.running_var.detach() + self.momentum * var.squeeze()
        else:
            mean = self.running_mean.view(1, self.num_features, 1, 1)
            var = self.running_var.view(1, self.num_features, 1, 1)
            x_normalized = (x - mean) / torch.sqrt(var + self.eps)
        return self.gamma.view(1, self.num_features, 1, 1) * x_normalized + self.beta.view(1, self.num_features, 1, 1)


class MyInstanceNorm2d(nn.Module):
    def __init__(self, num_features, eps=1e-5):
        super(MyInstanceNorm2d, self).__init__()
        self.eps = eps
        self.num_features = num_features  
        self.gamma = nn.Parameter(torch.ones(1, num_features, 1, 1))
        self.beta = nn.Parameter(torch.zeros(1, num_features, 1, 1))

    def forward(self, x):
        mean = x.mean(dim=[2, 3], keepdim=True)
        var = x.var(dim=[2, 3], keepdim=True, unbiased=False)
        x_normalized = (x - mean) / (torch.sqrt(var + self.eps))
        return x_normalized * self.gamma + self.beta
    
    
    
import json
import os
import copy
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import f1_score, accuracy_score
from tqdm import tqdm

def train_and_evaluate_model(model, train_loader, val_loader, test_loader, criterion, optimizer, num_epochs=50, device='cuda', patience=3, save_path='model.pth'):
    model.to(device)
    best_acc = 0.0
    best_model_wts = copy.deepcopy(model.state_dict())
    history = {'train_loss': [], 'val_loss': [], 'test_loss': [],
               'train_acc': [], 'val_acc': [], 'test_acc': [],
               'train_f1_micro': [], 'val_f1_micro': [], 'test_f1_micro': [],
               'train_f1_macro': [], 'val_f1_macro': [], 'test_f1_macro': []}
    epochs_no_improve = 0  # For early stopping
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        
        for phase in ['train', 'val', 'test']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            data_loader = train_loader if phase == 'train' else val_loader if phase == 'val' else test_loader
            
            running_loss = 0.0
            running_corrects = 0
            all_preds = []
            all_labels = []
            
            for inputs, labels in tqdm(data_loader, desc=f"{phase} Epoch {epoch+1}"):
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                all_preds.extend(preds.view(-1).cpu().numpy())
                all_labels.extend(labels.view(-1).cpu().numpy())
            
            epoch_loss = running_loss / len(data_loader.dataset)
            epoch_acc = running_corrects.double() / len(data_loader.dataset)
            epoch_f1_micro = f1_score(all_labels, all_preds, average='micro')
            epoch_f1_macro = f1_score(all_labels, all_preds, average='macro')
            
            history[f'{phase}_loss'].append(epoch_loss)
            history[f'{phase}_acc'].append(epoch_acc.item())
            history[f'{phase}_f1_micro'].append(epoch_f1_micro)
            history[f'{phase}_f1_macro'].append(epoch_f1_macro)
            
            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f} F1 Micro: {epoch_f1_micro:.4f} F1 Macro: {epoch_f1_macro:.4f}")
            
            if phase == 'val' and epoch_acc>best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model.state_dict(), save_path)
                epochs_no_improve = 0
            elif phase == 'val':
                epochs_no_improve += 1
            
            if epochs_no_improve == patience:
                print("Early stopping")
                break
        
        if epochs_no_improve == patience:
            break
    
    model.load_state_dict(best_model_wts)
    save_metrics(history, save_path + '_metrics.json')
    return model, history

def save_metrics(history, file_name):
    with open(file_name, 'w') as f:
        json.dump(history, f, indent=4)


In [3]:

model = ResNet18(n=2, num_classes=25, norm_layer=lambda num_features: MyNoNorm(num_features))

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define save path for the model and metrics
model_save_path = '/kaggle/working/resnet18_no_norm_32.pth'

# Train and evaluate the model
trained_model, history = train_and_evaluate_model(
    model=model,
    train_loader=train_data_loader,
    val_loader=val_data_loader,
    test_loader=test_data_loader,
    criterion=criterion,
    optimizer=optimizer,
    num_epochs=50,  
    device=device,
    patience=5,  # Early stopping patience
    save_path=model_save_path
)

# After training, metrics will be saved to 'resnet18_batchnorm_metrics.json'


Epoch 1/50


train Epoch 1: 100%|██████████| 920/920 [04:23<00:00,  3.49it/s]


train Loss: 2.8510 Acc: 0.1154 F1 Micro: 0.1154 F1 Macro: 0.1034


val Epoch 1: 100%|██████████| 263/263 [00:58<00:00,  4.50it/s]


val Loss: 2.5877 Acc: 0.1819 F1 Micro: 0.1819 F1 Macro: 0.1441


...

test Loss: 0.8693 Acc: 0.7995 F1 Micro: 0.7995 F1 Macro: 0.8008
Epoch 43/50


train Epoch 43: 100%|██████████| 920/920 [02:32<00:00,  6.02it/s]


train Loss: 0.3240 Acc: 0.8953 F1 Micro: 0.8953 F1 Macro: 0.8956


val Epoch 43: 100%|██████████| 263/263 [00:24<00:00, 10.69it/s]

val Loss: 0.4041 Acc: 0.8761 F1 Micro: 0.8761 F1 Macro: 0.8760
Early stopping


In [10]:
model = ResNet18(n=2, num_classes=25, norm_layer=lambda num_features: MyNoNorm(num_features))
model_path = "G:/inference/models/part_1.2_nn.pth"

# Load the model
state_dict = torch.load(model_path)

# Load the state dictionary into the model
model.load_state_dict(state_dict)


# Set the model to evaluation mode
# model.eval()

<All keys matched successfully>

In [4]:
model

OrderedDict([('conv1.weight',
              tensor([[[[ 0.1732, -0.0609, -0.3380],
                        [ 0.1326, -0.0162, -0.2749],
                        [-0.1487, -0.1011,  0.1027]],
              
                       [[ 0.2988,  0.3691,  0.2669],
                        [ 0.1944,  0.2269,  0.0072],
                        [-0.0364, -0.2057, -0.0348]],
              
                       [[ 0.1281,  0.2251,  0.1563],
                        [-0.1103, -0.1773,  0.0789],
                        [-0.3129, -0.3190, -0.2193]]],
              
              
                      [[[ 0.0940, -0.0259, -0.0500],
                        [-0.2922,  0.0099,  0.3466],
                        [-0.0687,  0.1538,  0.1970]],
              
                       [[-0.2332, -0.0228, -0.0110],
                        [-0.4598,  0.0638, -0.0954],
                        [-0.3726, -0.2227,  0.0990]],
              
                       [[-0.0030, -0.0907,  0.0615],
                        [-